# Прогнозирование оттока клиентов банка
---

   


#### Постановка бизнес-задачи, описание предметной области. <a name="first-stage" id="first-stage"></a>  

##### Предметная область
Рассматривается задача прогнозирования оттока клиентов банка. Отток (churn) - прекращение использования клиентом банковских услуг. Удержание клиентов - приоритет для бизнеса из-за высокой стоимости их привлечения.  


##### Контекст бизнеса

**Банки стремятся**:
- Выявлять клиентов с высоким риском ухода
- Повышать уровень удержания
- Персонализировать предложения

**Ключевые аспекты:**
- Своевременное выявление оттока
- Оптимизация маркетинговых и удерживающих кампаний
- Увеличение прибыли за счёт лояльных клиентов



##### Бизнес-задача
Бизнес-задача – на основе имеющихся данных о клиентах проанализировать датасет для выявления причин ухода клиентов.

**Цель:**
- Снизить отток клиентов
- Поддержки стабильной прибыли банков
- Улучшения стратегий взаимодействия с клиентами


##### Описание набора данных
**Источник:** Kaggle - [Bank Customer Churn Prediction](https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction)  
**Объём:** ~10 000 клиентов  
**Целевая переменная:** `Exited` (1 — клиент ушёл, 0 — остался)

**Основные признаки**

| Признак                | Тип            | Описание                                                  |
|:-                      |:-:             |:-                                                         |
| `CustomerId`           | Integer        | Уникальный идентификатор клиента                          |
| `Surname`              | String         | Фамилия клиента                                           |
| `CreditScore`          | Integer        | Кредитный рейтинг клиента                                 |
| `Geography`            | String         | Страна проживания                                         |
| `Gender`               | String         | Пол клиента                                               |
| `Age`                  | Integer        | Возраст клиента                                           |
| `Tenure`               | Integer        | Лет сотрудничества                                        |
| `Balance`              | Float          | Остаток на счёте                                          |
| `NumOfProducts`        | Integer        | Кол-во продуктов банка                                    |
| `HasCrCard`            | Integer        | Наличие кредитной карты                                   |
| `IsActiveMember`       | Float          | Активность клиента                                        |
| `EstimatedSalary`      | Float          | Предполагаемый доход                                      |
| `Exited`               | Integer        | Ушел ли клиент или нет (целевая переменная)               |


##### Условные договоренности
1. Будем считать, что кредитный рейтинг >=750 - это хороший кредитный рейтинг
2. Будем считать, что кредитный рейтинг <=500 - это низкий кредитный рейтинг
3. Будем считать, что клиенты с балансом >=200 000 - это особо важные (VIP) клиенты
---

#### Предобработка данных <a name="content-1" id="content-1"></a>

In [ ]:
import pandas as pd
import numpy as np

##### Знакомство с данными <a name="content-11" id="content-11"></a>

In [ ]:
# Загрузка дата сета
dataset = pd.read_csv("../data/churn.csv")
dataset

In [ ]:
# первые 10 строк
dataset.head(10)

In [ ]:
# последние 10 строк
dataset.tail(10)

In [ ]:
# случайные 10 строк
dataset.sample(10)

In [ ]:
# название колонок в датасете
dataset.columns

In [ ]:
# размер датасета
dataset.index

In [ ]:
# Общие сведения о датасете 
dataset.info()

##### Проблемы при беглом анализе <a name="content-12" id="content-12"></a>
При беглом осмотре можно сразу заметить несколько проблем в данном датасете:  
1. Колонка RowNumber - по сути является счетчиком строк, что не нужно, т.к порядок строки в данных не влияет на результат
2. Колонка CustomerId - уникальный идентификатор клиента, по сути не влияет на результат
3. В колонке Age значения представлены в виде чисел с плавающей точкой (float), хотя возраст исчисляется целочисленно
4. Колонки HasCrCard и IsActiveMember, возможно, содержат числа с плавающей точкой. В описании данных четко указано, что значениями могут быть только 1 (Да) и 0 (Нет)
5. Колонка Gender, возможно, имеет только 2 значения - Male и Female, можно ввести ассоциацию: 1 (Male) и 2 (Female)
6. Колонка Geography имеет только 3 значения - можно ввести ассоциацию: 1 (France), 2 (Spain), 3 (Germany)
7. Были замечены пустые значения
8. Названия колонок не соответствуют "змеиному регистру"
9. У некоторых имен имеются спец символы
10. У некоторых фамилий обнаружены спец. символы (например H? в строке с индексом 9)

##### Детальный анализ датасета и каждого столбца <a name="content-13" id="content-13"></a>

In [ ]:
# типы данных каждого столбца
dataset.dtypes

In [ ]:
# точное измерение памяти
dataset.memory_usage(deep=True)

In [ ]:
# поиск пустых значений по столбцам
dataset.isnull().sum()

In [ ]:
# детальный анализ каждого столбца

for column in dataset.columns:
    print(f"Колонка {column}")
    print(f"Тип данных: {dataset[column].dtype}")
    print(f"Количество пустых значений: {dataset[column].isnull().sum()}")
    print(f"Количество уникальных значений: {dataset[column].nunique()}")
    print(f"Уникальные значения: {dataset[column].unique()}")
    print(f"{dataset[column].describe()}")
    print()

При детальном анализе выявлены пустые значения, а также обнаружены столбцы, где можно привести данные в нормальную форму, например в колонке Age нужно привести к типу int.  
Также было определено, что колонки Geography, Gender являются категориальными - это значит, что тип данных object можно привести к int с помощью ассоциаций.

##### Исправление датасета <a name="content-14" id="content-14"></a>
Сначала обработаем регистры в столбцах и в значениях столбцов

In [ ]:
# Исправление регистров столбцов и приведение к "змеиному регистру"
dataset.columns = (
    dataset.columns
    .str.replace(r'(?<=[a-z])([A-Z])', r'_\1', regex=True)
    .str.lower()
)
dataset.columns.to_list()

Далее удалим столбцы row_number и customer_id, surname, так как ими можно пренебречь

In [ ]:
dataset = dataset.drop(["row_number", "customer_id", "surname"], axis=1)

Для столбцов gender и geography изменим регистр

In [ ]:
# замена значений на ассоциации
dataset[["gender", "geography"]] = dataset[["gender", "geography"]].apply(lambda x: x.str.lower())

Обратим внимание, что в колонке geography неверно установился тип данных  
Возможно это связано с наличием пропусков в колонке. Далее будет рассмотрена обработка пропусков и преобразование типов данных

##### Обработка пропусков <a name="content-15" id="content-15"></a>
В датасете было обнаружено 4 пропуска:

In [ ]:
rows_with_missing = dataset[dataset.isna().any(axis=1)]
rows_with_missing

Для каждой замены необходимо выбрать те методы, которые сохранят статические свойства данных (в случае если заполнить их не представляется возможным - удалить):
1. Для категориального значения в столбце geography воспользуемся модой (наиболее часто встречающееся значение). Данный способ сохраняет распределение категориальных признаков
2. Для значения в столбце age воспользуемся медианой. Медиана устойчива к выбросам (в отличие от среднего), что важно для возраста, где могут быть аномально высокие или низкие значения.
3. Если информация о наличии кредитной карты отсутствует (в столбце has_cr_card), разумно предположить, что её нет (более консервативный подход). Альтернатива — заполнение модой, но это может исказить данные, если большинство клиентов имеют карту. Поэтому заполним значение вручную
4. Для значения в столбце is_active_member для минимизации искажения воспользуемся модой, т.к мода сохраняет доминирующую категорию без искажений

In [ ]:
# geography 
dataset.loc[6, 'geography'] = dataset['geography'].mode()[0]

# age
dataset.loc[9, 'age'] = dataset['age'].median()

# has_cr_card
dataset.loc[4, 'has_cr_card'] = 0

# is_active_member
dataset.loc[8, 'is_active_member'] = dataset['is_active_member'].mode()[0]

# перепроверка пропусков
dataset.isnull().sum()

##### Приведение значений столбцов к необходимым типам данных <a name="content-16" id="content-16"></a>
В ходе анализа датасета установлено, что необходимо изменить следующие типы данных:
1. credit_score с int64 на int32 (оптимизация памяти)
2. geography не меняем
3. gender не меняем
4. age с float на int8 (изначально неверный тип данных)
5. tenure c int64 на int8 (оптимизация памяти)
6. num_of_products с int32 на int16 (оптимизация памяти)
7. has_cr_card с float на bool (неверный тип данных)
8. is_active_member c float на bool (неверный тип данных)
9.  exited с int на bool (неверный тип данных)
    
В основном изменения типов данных нужны для оптимизации памяти, но присутствуют и неверные типы данных

>Также нужно учесть особенности работы pandas с работой чисел с плавающей точкой (float):  
>float64 является стандартным представлением чисел с плавающей точкой в pandas и в начальных настройках отображения данных автоматически показывает только 2 знака после точки, хотя float64 имеет точность ~16 знаков после запятой. Изменение данного типа данных на float32 может привести к большему накоплению ошибок, чем float64.  
>Для денежных форматов необходимо использовать тип данных Decimal, но т.к датасет носит учебный характер - можно оставить float64.

In [ ]:
dataset = dataset.astype({
    'credit_score': 'int32',
    'age': 'int8',
    'tenure': 'int8',
    'num_of_products': 'int16',
    'has_cr_card': 'bool',
    'is_active_member': 'bool',
    'exited': 'bool'
})

# проверка отработало ли преобразование типов корректно или нет
dataset.dtypes

##### Поиск дубликатов <a name="content-17" id="content-17"></a>

In [ ]:
# Поиск дубликатов
dataset.duplicated().sum()

In [ ]:
# т.к нашлись дубликаты, выведем их
duplicates = dataset[dataset.duplicated(keep=False)]
duplicates

На печати видно, что дубликаты полностью идентичны (9998 с 9999 и 10000 с 10001).
Это может быть связано с разными причинами, например, ошибка при выгрузке датасета (на api или бд могли произойти сбои, которые вызвали повторную запись - это маловероятно, но не равно 0). Также стоит учесть, что это датасет для обучающихся и дублирование данных могло произойти нарочно.  
Также стоит учесть, что могло произойти некорректное объединение нескольких датасетов в один.  
Для устранения дублирования удалим дубликаты с сохранением одного экземпляра.

In [ ]:
dataset = dataset.drop_duplicates()
dataset.duplicated().sum()

##### Проверка датасета после предобработки данных <a name="content-18" id="content-18"></a>

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.memory_usage(deep=True)

##### Вывод по первому этапу <a name="content-19" id="content-19"></a>

В ходе предварительного анализа и обработки набора данных были выполнены следующие шаги и устранены выявленные проблемы:

1. Были просмотрены первые и последние строки, случайная выборка, структура датасета, типы данных и использование памяти.  
   Это позволило сформировать общее представление о содержимом датасета и выявить потенциальные проблемы: наличие ненужных столбцов, неунифицированные имена, категориальные признаки в строковом формате, пропуски в данных, дублирование данных и несоответствие типов данных.

2. При анализе датасета были выявлены:
   - Наличие неинформативных столбцов (`row_number`, `customer_id`), которые были удалены из датасета;
   - Несоблюдение стиля наименования столбцов (`"змеиного регистра"`). В процессе обработки все строковые значения (включая название столбцов) были приведены к этому стилю;
   - Наличие строкового формата у категориальных признаков (`gender`, `geography`). Для решения данной проблемы были созданы отдельные словари, а в датасете использованы цифровые ключи;
   - Наличие пустых значений и дубликатов, которые в процессе были устранены;
   - Несоответствие типов данных. После предобработки все данные были приведены к необходимым типам

3. Столбец `surname` был очищен от специальных символов и приведён к нижнему регистру, после чего был закодирован с помощью pd.factorize, что позволило превратить строковый столбец в числовой идентификатор.
   
Таким образом данные были подготовлены к дальнейшему анализу.


#### Исследовательский анализ данных


##### Индексация данных

В данном пункте рассматриваются 2 вида индексаций: по координатам и логическая.  
Индексация по координатам осуществляет доступ к данным по их позиции в датасете. Используется, когда нужно обратиться к данным по их номерам строк и столбцов (чаще нужно для срезов).  
Логическая индексация осуществляет доступ к данным по определенным условиям. Используется для выборки данных по условиям.

Для индексации по координатам проанализируем определенные ситуации, когда необходимо взять данные по их позиции:
1. Выделим только определенные финансовые показатели клиентов (кредитный рейтинг, баланс, наличие кредитной карты и предполагаемую зарплату)
2. Выделим только демографические признаки клиентов (столбцы geography, gender, age)
3. Выделим данные для анализа зависимости возраста, количества продуктов и баланса 
4. Создадим выборочные данные для проверки с помощью выделения каждого 20 клиента
5. Выделим определенный диапазон клиентов (строк). Предположим, что в конце датасета содержится информация о новых клиентах

(у меня не хватило фантазии для такого типа индексации)

In [ ]:
'''
№ 1
Сделаем выборку для анализа платежеспособности клиентов
с помощью выделения финансовых показателей
Для этого берем у всех клиентов только те столбцы, которые могут дать информацию о платежеспособности
'''
financial_indicators = dataset.iloc[:, [1, 6, 8, 10]]
financial_indicators

In [ ]:
'''
№ 2
Сделаем выборку для анализа по демографическому признаку
Для этого берем у всех клиентов столбцы о географическом положении, гендее и возрасте
'''
demographics = dataset.iloc[:, [2, 3, 4]]
demographics


In [ ]:
'''
№ 3
Сделаем выборку для проверки зависимости возраста и баланса на количество продуктов в банке
'''
age_num_prod = dataset.iloc[:, [4, 6, 7]]
age_num_prod

In [ ]:
'''
№ 4
Сделаем выборочные данные, например, для оценки репрезентативности данных
Для этого выберем каждого 20 клиента из датасета
'''
selection_data = dataset.iloc[::20, :]
selection_data

In [ ]:
'''
№ 5
Предположим, что в конце датасета содержится информация о новых клиентах (т.е первые строки - это клиенты, которые были зарегистрированы в базе банка раньше)
Для этого выберем последние 2000 клиентов из датасета 
'''
new_clients = dataset.iloc[-2000:, :]
new_clients

Для логической индексации проанализируем следующие ситуации:
1. Выделим клиентов пожилого возраста (старше 60 лет)
2. Выделим клиентов с высоким кредитным рейтингом и низким балансом
3. Выделим клиентов, которые ушли по гендеру
4. Выделим клиентов с 2я и более продуктами, но без кредитной карты
5. Выделим клиентов, которые ушли в первый год обслуживания в банке

In [ ]:
'''
№ 1
Сделаем выборку для анализа активности клиентов пожилого возраста
'''
oldest_clients = dataset[dataset["age"] >= 60]
oldest_clients

In [ ]:
'''
№ 2
Сделаем выборку для поиска клиентов с высоким кредитным рейтингом (будем считать, что 750 это высокий кредитный рейтинг), но с низким балансом
'''
strange_behavior = dataset[(dataset["credit_score"] >= 750) & (dataset["balance"] <= 1000)]
strange_behavior

In [ ]:
'''
№ 3.1
Разделим ушедших клиентов по гендеру для поиска причин оттока (м)
'''
male_churn = dataset[(dataset["gender"] == "male") & (dataset["exited"] == True)]
male_churn

In [ ]:
'''
№ 3.2
Разделим ушедших клиентов по гендеру для поиска причин оттока (ж)
'''
female_churn = dataset[(dataset["gender"] == "female") & (dataset["exited"] == True)]
female_churn

In [ ]:
'''
№ 4
Выделим клиентов с 2 и более продуктами, но без кредитной карты для определения потенциальных кандидатов на предложение кредитной карты
для такого анализа также важно, чтобы клиенты были активными
'''
credit_offer = dataset[(dataset["num_of_products"] >= 2) & (dataset["has_cr_card"] == False) & (dataset["is_active_member"] == True)]
credit_offer

In [ ]:
'''
№ 5
Выделим клиентов, которые ушли в первый год обслуживания (т.к tenure у нас целочисленное, то будем считать клиентов с tenure<=1)
'''
departed_new_clients = dataset[(dataset["tenure"] <= 1) & (dataset["exited"] == True)]
departed_new_clients

##### Сортировка данных
Благодаря сортировки данных мы можем проанализировать максимальные и минимальные значения данных для выявления групп риска.  
Выполним сортировку по следующим столбцам: 
1. По кредитному рейтингу
2. По возрасту
3. По балансу
4. По количеству продуктов
Также выполним одну многоуровневую сортировку для выявления новых клиентов с высоким кредитным рейтингом, но которые ушли из банка  

>Параметр `ascending` определяет порядок сортировки (True - по возрастанию, False - по убыванию)

In [ ]:
'''
№ 1 Сортировка по кредитному рейтингу
Благодаря данной сортировки мы можем взять с помощью индексации по координатам клиентов с низким рейтингом и высоким и отдельно их проанализировать
'''
dataset_sorted_credit = dataset.sort_values("credit_score", ascending=False)
dataset_sorted_credit

In [ ]:
# дополнительный вывод строк, тк видна связь между рейтингом и исходом
dataset_sorted_credit.tail(20)

Благодаря сортировке по кредитному рейтингу удалось выделить клиентов с максимальным и минимальным возможным в данном датасете кредитном рейтинге.  
Из этих данных можно увидеть, что почти все клиенты с низким рейтингом имеют кредитную карты, а также топ-20 клиентов с низким рейтингом покинули банк.  
Также стоит отметить, что топ-5 клиентов по кредитному рейтингу остаются в банке.  
Таким образом можно сделать вывод, что клиенты с низким кредитным рейтингом являются группой с высоким риском ухода из банка.

In [ ]:
'''
№ 2 Сортировка по возрасту
Благодаря данной сортировки можно разделить клиентов по возрастным группам, разделив датасет на несколько частей для определения стратегий работы с каждой вохрастной группой отдельно
'''
dataset_sorted_age = dataset.sort_values("age", ascending=False)
dataset_sorted_age

Анализируя данные стоит отметить, что крайние возрастные группы не проявляют высокого оттока. Это значит, что нужно более детально проанализировать клиентов среднего возраста. Также стоит отметить, что баланс не всегда влияет на отток - и молодые и старшие клиенты с нулевым балансом остаются в банке.

In [ ]:
'''
№ 3 Сортировка по балансу
Благодаря данной сортировки можно разделить людей с очень высоким балансом и очень низким
'''
dataset_sorted_balance = dataset.sort_values("balance", ascending=False)
dataset_sorted_balance

Исходя из данных можно сделать вывод:  
Клиенты с очень высоким остатком на счёте (>= 200 000) нужно расценивать как VIP клиентов. Если среди них есть те, кто ушёл из банка, это тревожный сигнал, т.к такие клиенты ценны для бизнеса. При анализе видно, что клиенты с самым высоким балансом ушли из банка, а значит банку необходимо пересмотреть стратегию работы с VIP клиентами.  
Также интересны клиенты с нулевым балансом — это может быть как неактивный клиент, так и клиент, использующий только кредитные продукты.

In [ ]:
'''
№ 4 Сортировка по количеству продуктов
Благодаря данной сортировки можно явно выделить клиентов с большим количеством продуктов и тех клиентов, которые вообще не имеют продуктов (но в данном датасете таких нет)
'''
dataset_sorted_num_prod = dataset.sort_values("num_of_products", ascending=False)
dataset_sorted_num_prod

In [ ]:
# дополнительный вывод, тк замечена связь между максимальным количеством продуктов и исходом
dataset_sorted_num_prod.head(20)

Исходя из данной сортировки можно наблюдать следующее:  
Клиенты, у которых больше всего банковских продуктов (4), гораздо чаще уходят, чем те, у кого 1 продукт.  
Из данного наблюдения можно сформировать следующий риск - риск ухода возрастает с увеличением числа продуктов. Это может быть связано с плохим уровнем поддержки продуктов (банк продал 4 продукта, которые плохо взаимодействуют между собой, что могло вызвать недовольство клиентов)


In [ ]:
'''
№ 5 Многоуровневая сортировка для выявления новых клиентов с высоким кредитным рейтингом, но которые ушли из банка  
Благодаря данной сортировки можно выявить клиентов для дальнейшего поиска причин ухода
'''
dataset_sorted_multi_level = dataset.sort_values(["tenure", "exited", "credit_score"], ascending=[True, False, False])
dataset_sorted_multi_level

In [ ]:
# дополнительный вывод, тк замечена связь
dataset_sorted_multi_level.head(20)

Из анализа видно, что новые клиенты (до 1 года обслуживания в банке) с высоким кредитным рейтингом ушли.  
Это очень важный сигнал для бизнеса - банк теряет новых, надежных (с точки зрения кредитного рейтинга) клиентов.  
Данная проблема может быть связана с:
1. Сервисы банка не соответствуют ожиданию;
2. Нет нормальной поддержки продуктов
3. Отсутствие персонального сопровождения.


Из всего текущего анализа можно выделить основные группы риска - клиенты с низким кредитным рейтингом (=<500) и новые клиенты с высоким рейтингом, уходящие в первый год, что может указывать на слабую конкурентноспособность анализируемой банковской системы. Особенно нужно обратить внимание на уход VIP-клиентов (баланс >=200 000) и клиентов с 4+ продуктами - вероятно из-за сложного обслуживания или более выгодных условий конкурентов.  
На текущий момент можно сделать следующее `предположение`:  
>Текущий анализ данных показывает картину консервативной банковской системы в 3х представленных странах. Данная банковская система недооценивала важность персональных услуг и длительное время работала по универсальным шаблонам. Особенно это видно по уходу VIP клиентов (те, у которых баланса >200 000). В такой ситуации банковской системе срочно нужно перестраиваться и улучшать свои сервисы

##### Фильтрация данных с помощью query и where

`query` используйте для сложных условий в одну строку  
Через query определим следующие данные:
1. Клиенты с низким кредитным рейтингом, которые ушли
2. Клиенты с высоким балансом, которые ушли
3. Новые клиенты, которые ушли
4. Клиенты с 4 банковскими продуктами, которые ушли
5. Активные клиенты с нулевым балансом

`where` применяйте для категоризации и сегментации  
Через where определим следующие данные:
1. Клиенты с высоким кредитным рейтингом
2. Пожилые клиенты с высоким балансом
3. Клиенты с высоким доходом, но неактивные
4. Клиенты, ушедшие в первые 2 года
5. Мужчины и женщины (отдельно) с 1 банковским продуктом

In [ ]:
# 1.1 - Клиенты с низким кредитным рейтингом, которые ушли
low_credit_score = dataset.query("credit_score <= 500 and exited == True")
low_credit_score

In [ ]:
# 1.2 - Клиенты с высоким балансом, которые ушли
high_balance_churned = dataset.query("balance >= 200000 and exited == True")
high_balance_churned

In [ ]:
# 1.3 - Новые клиенты, которые ушли
new_clients_churned = dataset.query("tenure <= 1 and exited == True")
new_clients_churned

In [ ]:
# 1.4 - Клиенты с 4 продуктами, которые ушли
multi_product_churned = dataset.query("num_of_products == 4 and exited == True")
multi_product_churned

In [ ]:
# 1.5 - Активные клиенты с нулевым балансом
active_zero_balance = dataset.query("balance == 0 and is_active_member == True")
active_zero_balance

In [ ]:
# Для метода numpy.where скопируем датасет и будем работать с копией
dataset_copy = dataset.copy()

In [ ]:
# 2.1 - Клиенты с высоким кредитным рейтингом
dataset_copy["high_credit"] = np.where(dataset_copy["credit_score"] > 750, "high", "normal")
high_credit_clients = dataset_copy[dataset_copy["high_credit"] == "high"]
high_credit_clients

In [ ]:
# 2.2 - Пожилые клиенты с высоким балансом
dataset_copy["senior_vip"] = np.where(
    (dataset_copy["age"] > 60) & (dataset_copy["balance"] > 100000), 
    "vip", 
    "other"
)
senior_vip = dataset_copy[dataset_copy["senior_vip"] == "vip"]
senior_vip

In [ ]:
# 2.3 - Клиенты с высоким доходом, но неактивные
dataset_copy["rich_inactive"] = np.where(
    (dataset_copy["estimated_salary"] > 100000) & (dataset_copy["is_active_member"] == False),
    "rich_inactive",
    "active"
)
inactive_rich = dataset_copy[dataset_copy["rich_inactive"] == "rich_inactive"]
inactive_rich

In [ ]:
# 2.4 - Клиенты, ушедшие в первые 2 года
dataset_copy["early_churn"] = np.where(
    (dataset_copy["tenure"] <= 2) & (dataset_copy["exited"] == True),
    "early_churn",
    "loyal"
)
early_churners = dataset_copy[dataset_copy["early_churn"] == "early_churn"]
early_churners

In [ ]:
# 2.5 - Мужчины и женщины с 1 продуктом
dataset_copy["gender_one_product"] = np.where(
    (dataset_copy["num_of_products"] == 1),
    dataset_copy["gender"].map({"male": "male_one_product", "female": "female_one_product"}), "other"
)
male_one_product = dataset_copy[dataset_copy["gender_one_product"] == "male_one_product"]
female_one_product = dataset_copy[dataset_copy["gender_one_product"] == "female_one_product"]

male_one_product

In [ ]:
female_one_product

##### Составление сводных таблиц

В рамках данной задачи сформируем 3 сводные таблицы:
1. Отток по гендеру и наличию кредитной карты

In [ ]:
pivot_gender_card = dataset.pivot_table(
    index='gender',
    columns='has_cr_card',
    values='exited',
    aggfunc='mean',
    margins=True
)
pivot_gender_card

По данной сводной таблице можно заметить значительный гендерный разрыв в оттоке клиентов: женщины уходят на 50% чаще мужчин (25.1% против 16.5%). Наличие кредитной карты снижает отток лишь незначительно (на 0.5-0.6%)

2. Отток по активности и количеству продуктов

In [ ]:
pivot_activity_products = dataset.pivot_table(
    index='is_active_member',
    columns='num_of_products',
    values='exited',
    aggfunc='count'
)

pivot_activity_products = pivot_activity_products.rename(index={0: "inactive", 1: "active"})
pivot_activity_products

Основной количество клиентов (около 5000) имеют 1-2 продукта. Клиентов с 3-4 продуктами значительно меньше (всего 326), причем активных пользователей среди них крайне мало (113 и 29 соответственно)

3. Средние баланс и зарплата ушедших/оставшихся клиентов

In [ ]:
pivot_balance_salary = dataset.pivot_table(
    index='exited',
    values=['balance', 'estimated_salary'],
    aggfunc='mean'
)
pivot_balance_salary = pivot_balance_salary.rename(index={0: "stayed", 1: "left"})
pivot_balance_salary

Из этих данных можно заметить, что предполагаемая зарплата ушедших и оставшихся клиентов почти одинаковая, но значительно (на 18 363, что примерно 20%) отличается баланс.  
Можно предположить, что оставшиеся клиенты больше тратят, а ушедшие предпочитают копить, но для такого предположения необходимо проанализировать количество транзакций, которых в данном датасете нет. 

##### Группировка данных

Исходя из предыдущего анализа, выделим следующие группы и вычислим агрегатные функции:
1. Клиенты с низким кредитным рейтингом
2. Новые клиенты с высоким кредитным рейтингом
3. Ушедшие VIP-клиенты
4. Клиенты с 3+ продуктами
   
В данном анализе используются следующие агрегатные функции:
- `mean` – среднее арифметическое значение
- `median` – возвращает значение, которое делит данные пополам (50-й перцентиль)
- `min` – минимальное значение
- `max` – максимальное значение
- `count` – количество непустых значений
- `std` – стандартное отклонение
- `var` – дисперсия
- `skew` - коэффициент асимметрии


In [ ]:
# 1 Клиенты с низким кредитным рейтингом
low_credit_stats = dataset[dataset["credit_score"] <= 500].groupby("exited").agg({
    "age": "median",
    "balance": ["mean", "median", "count", "std", "skew"],
    "num_of_products": ["mean", "var"],
    "has_cr_card": "mean",
    "tenure": ["min", "max", "mean"],
    "estimated_salary": "median"
}).round(2)
low_credit_stats

Анализ клиентов с низким кредитным рейтингом (≤500) показал, что ушедшие клиенты в среднем старше (медианный возраст 46 против 35 лет) и имеют более высокие финансовые показатели: их средний баланс на 16,8% выше, а медианная зарплата — на 17,4%. При этом распределение количества продуктов у них более неравномерное (дисперсия в 2,5 раза выше), что указывает на две ключевые группы риска: клиенты с одним продуктом (низкая вовлеченность) и клиенты с тремя и более продуктами (потенциальная перегруженность или отсутствие нормальной поддержки продуктов).

In [ ]:
# 2 Новые клиенты с высоким кредитным рейтингом (группируем сразу по 2ум признакам - гендер и исход)
new_high_credit_stats = dataset[(dataset["credit_score"] >= 750) & 
                              (dataset["tenure"] <= 1)].groupby(["exited", "gender"]).agg({
    "age": ["median"],
    "balance": ["mean", "median", "count", "std", "skew"],
    "num_of_products": ["mean", "var"],
    "estimated_salary": ["mean", "median", "count", "std", "skew"],
    "is_active_member": "mean"
}).round(2)
new_high_credit_stats

Анализ новых клиентов с высоким кредитным рейтингом (≥750) и сроком сотрудничества ≤1 год подтверждает, что ушедшие клиенты значительно старше (медианный возраст 47–48 лет против 36–37 у оставшихся) и имеют больший баланс (средний 102,735 у женщин и 84,124 у мужчин против 77,590–80,781 у лояльных). При этом их активность ниже (47% у женщин и 27% у мужчин против 61–62%), а зарплаты слабо коррелируют с оттоком.

Исходя из анализа уже двух групп можно сделать вывод:  
> Банк теряет финансово обеспеченных клиентов старшего возраста уже в первый год, вероятно, из-за несоответствия условий их потребностям.

In [ ]:
# 3 Ушедшие VIP-клиенты
vip_churned_stats = dataset[(dataset["balance"] >= 200000) & 
                          (dataset["exited"] == True)].groupby(["geography", "num_of_products"]).agg({
    "age": "median",
    "tenure": ["mean", "median", "std"],
    "credit_score": ["mean", "median", "min", "max", "count"],
    "is_active_member": "mean",
    "estimated_salary": ["mean", "median", "count", "std", "skew"]
}).round(2)
vip_churned_stats

При анализе участвуют VIP-клиенты из 2 регионов - Франция и Испания.  
Во Франции клиенты с 1м продуктом уходят в среднем после 5.4 лет (медианный возраст 43 года), несмотря на высокий кредитный рейтинг (в среднем 653)  
В Испании клиенты с 2я продуктами уходят в среднем после 3–4.4 года, даже при рейтинге 735+, что может указывать на системные проблемы в обслуживании  
Также стоит заметить низкую активность клиентов с 1-2 продуктами, но для точного утверждения слишком мало данных

In [ ]:
# 4 Клиенты с 3+ продуктами (группируем сразу по исходу, активности и наличию кредитных карт)
multi_product_stats = dataset[dataset["num_of_products"] >= 3].groupby(
    ["exited", "is_active_member", "has_cr_card"]).agg({
    "balance": ["mean", "median", "count", "std", "skew"],
    "age": "median",
    "tenure": ["mean", "median", "std"],
    "credit_score": ["mean", "median", "min", "max", "count"],
    "estimated_salary": ["mean", "median", "count", "std", "skew"]
}).round(2)
multi_product_stats

Ушедшие клиенты с 3+ продуктами имеют значительно больший баланс (средний 81–92K против 15–32K у оставшихся) и старше (медианный возраст 44–46 лет против 33–35).  
При этом их кредитный рейтинг ниже (в среднем 644–647 против 618–742). Можно сделать вывод, что клиенты с высоким балансом и 3+ продуктами уходят, несмотря на долгое сотрудничество (медианный срок 4–6 лет).

##### Свободный исследовательский анализ

В процессе выполнения пунктов 1-5 исследовательского анализа были выявлены следующие закономерности, которые могут указывать на риски оттока клиентов:
1. Клиенты, у которых 3 и более банковских продуктов, чаще уходят из банка. Это может быть связано с навязыванием услуг или перегруженностью предложениями, что вызывает раздражение и снижает лояльность.
2. Клиенты, которые обслуживались в банке 1 год или меньше, даже при высоком кредитном рейтинге, склонны к оттоку. Возможно, они не увидели преимуществ банка сразу и предпочли другие.
3. Отток наблюдается и среди клиентов с балансом свыше 200 000, что может указывать на отсутствие подходящих премиальных продуктов или неудовлетворительный сервис.

Однако, стоит обратить внимание на неактивных клиентов:

In [ ]:

random_churn = dataset.groupby(["geography", "gender" , "is_active_member", "exited"]).agg({
    "age": ["median", "mean"],
    "balance": ["mean", "median", "std", "count"],
    "credit_score": ["min", "max", "mean"],
    "num_of_products": ["mean", "std", "count"],
    "tenure": ["median", "mean", "max"],
    "estimated_salary": ["median", "mean", "std"]
}).round(2)
random_churn

Исходя из этих результатов можно заметить, что в германии клиенты в среднем имеют высокий баланс по сравнению с францией и испанией, но при этом эти клиенты уходят чаще, чем в других странах. Дополнительно проверим эту закономерность:

In [ ]:
# подсчет количества людей по странам
dataset['geography'].value_counts()

In [ ]:
# проверим отток клиентов с разным диапазоном баланса:
for balance in range(0, 120_000, 10_000):
    max_balance = balance*2 if balance != 0 else 10_000
    churn_by_country = dataset[(dataset['balance'] >= balance) & (dataset['balance'] <= max_balance)].groupby('geography')['exited'].mean().sort_values(ascending=False)
    print(f"Диапазон от {balance} до {max_balance}")
    print(churn_by_country)
    print()

In [ ]:
# отдельно проверим отток в определенном диапазоне баланса и количестве человек в этом диапазоне
range_balance = (dataset['balance'] >= 50_000) & (dataset['balance'] <= 100_000)
churn_mid_balance = dataset[range_balance].groupby('geography')['exited'].mean()
churn_mid_balance

In [ ]:
range_balance = dataset[(dataset['balance'] >= 50_000) & (dataset['balance'] <= 100_000)]
range_balance["geography"].value_counts()

Исходя из всех вышеперечисленных данных можно обнаружить следующую закономерность - в зависимости от диапазона дохода наблюдается отток клиентов по географическому признаку, а именно:
1. В диапазоне от 0 до 50 000 максимальный отток наблюдается среди клиентов из Испании и Франции, в то время как клиентов из Германии в этом диапазоне вообще нет
2. В диапазоне от 50 000 до 100 000 наблюдается рост оттока среди клиентов Германии
3. При балансе выше 70 000 Германия показывает самый высокий показатель оттока, который примерно в 2 раза больше по сравнению с Францией и Испанией
4. Самой стабильной страной является Франция за счет низкого оттока почти на всех диапазонах

##### Вывод по исследовательскому анализу

В ходе исследовательского анализа данных были изучены финансовые и демографические характеристики клиентов, выполнена фильтрация по ключевым бизнес-параметрам, проведены группировки и построены сводные таблицы. Это позволило выявить следующие важные закономерности:
- Отток чаще наблюдается у клиентов с несколькими банковскими продуктами, особенно при количестве более трех
- Новые клиенты, даже с хорошим кредитным рейтингом, не всегда остаются, что может указывать на слабое первое впечатление от банка. Если смотреть новых клиентов с хорошим кредитным рейтингом, то среди них отток составил почти 100%
- Клиенты с высоким балансом тоже склонны к оттоку
- Также выявлена неочевидная закономерность баланса и географического признака, которые влияют на отток

Таким образом, были определены группы клиентов, наиболее склонные к уходу. Эти выводы могут лечь в основу построения новой стратегий по удержанию клиентов.